In [22]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Embedding

# Import text_to_word_sequence from tf.keras.preprocessing.text
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from keras.utils import pad_sequences
from tensorflow.keras.utils import to_categorical

In [23]:
data = pd.read_csv('/content/drive/MyDrive/emoji_data.csv', header = None)
data.head()

,0,1
0,French macaroon is so tasty,4
1,work is horrible,3
2,I am upset,3
3,throw the ball,1
4,Good joke,2


In [24]:
emoji_dict = {
    0: ":red_heart:",
    1: ":baseball:",
    2: ":grinning_face_with_big_eyes:",
    3: ":disappointed_face:",
    4: ":fork_and_knife_with_plate:"
}

def label_to_emoji(label):
    return emoji.emojize(emoji_dict[label])

In [25]:
X = data[0].values
Y = data[1].values


In [27]:
file = open('/content/glove.6B.100d.txt', 'r', encoding = 'utf8')
content = file.readlines()
file.close()
embeddings = {}

for line in content:
    line = line.split()
    embeddings[line[0]] = np.array(line[1:], dtype = float)



In [29]:
import pandas as pd
import numpy as np
import emoji

# Assuming you are using a library like nltk for tokenization
import nltk
nltk.download('punkt') # Download punkt sentence tokenizer if not already present
from nltk.tokenize import word_tokenize

# ... (your existing code for reading data and defining functions) ...

X = data[0].values
Y = data[1].values

# Tokenize the text data in X
Xtokens = [word_tokenize(sent) for sent in X] # Tokenize each sentence in X

def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen, len(sent))
    return maxlen

maxlen = get_maxlen(Xtokens) # Now Xtokens is defined and contains tokenized sentences
print(maxlen)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


10


In [45]:
import pandas as pd
import numpy as np
import emoji
import nltk
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# ... (your existing code for reading data and defining functions) ...

X = data[0].values
Y = data[1].values

# Tokenize the text data in X
Xtokens = [word_tokenize(sent) for sent in X]

# ... (your existing code for get_maxlen) ...

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
word2index = tokenizer.word_index
Xtokens = tokenizer.texts_to_sequences(X)
Xtrain = pad_sequences(Xtokens, maxlen=maxlen, padding='post', truncating='post')

# Convert Y to integers if it contains non-integer values
# Assuming labels are '0', '1', '2', '3', '4', etc. or similar pattern
# You might need to adjust the logic based on your actual labels
def convert_to_int(label):
    try:
        return int(label)  # Try converting directly to int
    except ValueError:
        # If direct conversion fails, try removing non-digit characters
        # and converting the remaining part to int
        # For example, convert '3 ' to '3' and then to 3
        cleaned_label = ''.join(filter(str.isdigit, label))
        if cleaned_label:
            return int(cleaned_label)
        else:
            # Handle cases where label is completely invalid (e.g., "abc")
            # Replace with a default value or raise an error
            return -1 # or raise ValueError("Invalid label: {}".format(label))

Y_int = np.array([convert_to_int(label) for label in Y])

# Filter out invalid labels if any were found
valid_indices = np.where(Y_int != -1)[0]
Xtrain = Xtrain[valid_indices]
Y_int = Y_int[valid_indices]

Ytrain = to_categorical(Y_int)

In [47]:
embed_size = 100
embedding_matrix = np.zeros((len(word2index)+1, embed_size))

for word, i in word2index.items():
    # Check if the word is in the embeddings dictionary
    if word in embeddings:
        embed_vector = embeddings[word]
        embedding_matrix[i] = embed_vector
    else:
        # If not found, you can either:
        # 1. Initialize with random values:
        embedding_matrix[i] = np.random.uniform(-0.25, 0.25, embed_size)
        # 2. Skip the word:
        # pass
        # 3. Use a special token like <UNK> (if present in your embeddings)
        # if '<UNK>' in embeddings:
        #     embedding_matrix[i] = embeddings['<UNK>']

In [48]:
embedding_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [-0.046539,  0.61966 ,  0.56647 , ..., -0.37616 , -0.032502,
         0.8062  ],
       [-0.49886 ,  0.76602 ,  0.89751 , ..., -0.41179 ,  0.40539 ,
         0.78504 ],
       ...,
       [-0.46263 ,  0.069864,  0.69095 , ..., -0.29174 ,  0.32041 ,
         0.21202 ],
       [ 0.073242,  0.11134 ,  0.62281 , ...,  0.53417 , -0.1646  ,
        -0.27516 ],
       [ 0.29019 ,  0.80497 ,  0.31187 , ..., -0.33603 ,  0.45998 ,
        -0.11278 ]])

In [49]:
model = Sequential([
    Embedding(input_dim = len(word2index) + 1,
              output_dim = embed_size,
              input_length = maxlen,
              weights = [embedding_matrix],
              trainable = False
             ),

    LSTM(units = 16, return_sequences = True),
    LSTM(units = 4),
    Dense(5, activation = 'softmax')
])

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [50]:
model.fit(Xtrain, Ytrain, epochs = 100)

Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.2052 - loss: 1.5830
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2729 - loss: 1.5555 
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3059 - loss: 1.5219 
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2961 - loss: 1.5064 
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3298 - loss: 1.4940 
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3315 - loss: 1.4789 
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3207 - loss: 1.4769 
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3138 - loss: 1.4458 
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3336 - loss: 1.4421 
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3711 - loss: 1.3933 
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3263 - loss: 1.3650
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3995 - lo

In [53]:
test = ["I feel good", "What a lovely day", "I am very angry"]

test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen = maxlen, padding = 'post', truncating = 'post')

y_pred = model.predict(Xtest)
y_pred = np.argmax(y_pred, axis = 1)

for i in range(len(test)):
    print(test[i], label_to_emoji(y_pred[i]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
I feel good 😃
What a lovely day 😞
I am very angry 😞
